In [3]:
from web3 import Web3
import json
import os
import time
from eth_account import Account

# Base network configuration - using free public RPC with better rate limits
BASE_RPC_URL = "https://base.blockpi.network/v1/rpc/public"  # Free public endpoint
# Alternative: "https://base.meowrpc.com" or "https://base-pokt.nodies.app"
CHAIN_ID = 8453  # Base mainnet chain ID

# PokPok Protocol addresses - will be converted to checksum format in __init__
PEGG_TOKEN_ADDRESS = "0x82b0e1a2374ea0198f62a48b14ffab53db6c1e36"
# Note: You'll need to find the chicken NFT contract addresses
# Check your PokPok app transactions or BaseScan for the specific contract addresses
# CHICKEN_CONTRACT_ADDRESS = "0x..."  # Main chicken NFT contract
# GENESIS_FLOCK_CONTRACT = "0x..."    # Genesis Flock NFT contract

class PokPokInteractor:
    def __init__(self, private_key, rpc_url=BASE_RPC_URL):
        self.w3 = Web3(Web3.HTTPProvider(rpc_url))
        self.account = Account.from_key(private_key)
        self.address = self.account.address
        
        # Convert addresses to proper checksum format
        self.pegg_token_address = self.w3.to_checksum_address(PEGG_TOKEN_ADDRESS)
        
        # Verify connection with retry
        connected = False
        for attempt in range(3):
            try:
                if self.w3.is_connected():
                    connected = True
                    break
                else:
                    print(f"Connection attempt {attempt + 1} failed, retrying...")
                    time.sleep(1)
            except Exception as e:
                print(f"Connection attempt {attempt + 1} error: {e}")
                time.sleep(1)
        
        if not connected:
            raise Exception("Failed to connect to Base network after 3 attempts")
        
        print(f"Connected to Base network")
        print(f"Account address: {self.address}")
        
        # Get balance with retry
        try:
            balance = self.w3.eth.get_balance(self.address)
            print(f"Balance: {self.w3.from_wei(balance, 'ether')} ETH")
        except Exception as e:
            print(f"Could not get balance: {e}")
            
        print(f"PEGG Token Address: {self.pegg_token_address}")

    def get_pegg_balance(self):
        """Get PEGG token balance"""
        import time
        
        # ERC20 ABI for balanceOf function
        erc20_abi = [
            {
                "constant": True,
                "inputs": [{"name": "_owner", "type": "address"}],
                "name": "balanceOf",
                "outputs": [{"name": "balance", "type": "uint256"}],
                "type": "function"
            },
            {
                "constant": True,
                "inputs": [],
                "name": "decimals",
                "outputs": [{"name": "", "type": "uint8"}],
                "type": "function"
            }
        ]
        
        try:
            pegg_contract = self.w3.eth.contract(
                address=self.pegg_token_address,
                abi=erc20_abi
            )
            
            balance = pegg_contract.functions.balanceOf(self.address).call()
            time.sleep(0.2)  # Small delay to avoid rate limiting
            
            decimals = pegg_contract.functions.decimals().call()
            time.sleep(0.2)  # Small delay to avoid rate limiting
            
            return balance / (10 ** decimals)
            
        except Exception as e:
            print(f"Error getting PEGG balance: {e}")
            return 0

    def claim_pegg_from_chickens(self, chicken_contract_address, chicken_ids=None):
        """
        Claim PEGG from staked chickens - you'll need the actual contract ABI
        Based on PokPok docs: "healthy chickens can be staked in the farm to lay golden eggs ($PEGG)"
        """
        # Convert to checksum address
        chicken_contract_address = self.w3.to_checksum_address(chicken_contract_address)
        # You'll need to get the actual ABI from PokPok documentation or contract
        # Common function names might be: claimEggs, harvestEggs, claimRewards, etc.
        chicken_abi = [
            {
                "inputs": [{"name": "tokenIds", "type": "uint256[]"}],
                "name": "claimEggs",  # This might be the actual function name
                "outputs": [],
                "stateMutability": "nonpayable",
                "type": "function"
            }
        ]
        
        chicken_contract = self.w3.eth.contract(
            address=chicken_contract_address,
            abi=chicken_abi
        )
        
        # Build transaction
        if chicken_ids is None:
            chicken_ids = self.get_my_chicken_ids(chicken_contract_address)
        
        transaction = chicken_contract.functions.claimEggs(chicken_ids).build_transaction({
            'from': self.address,
            'gas': 200000,  # Adjust as needed
            'gasPrice': self.w3.eth.gas_price,
            'nonce': self.w3.eth.get_transaction_count(self.address)
        })
        
        # Sign and send transaction
        signed_txn = self.w3.eth.account.sign_transaction(transaction, private_key=self.account.key)
        tx_hash = self.w3.eth.send_raw_transaction(signed_txn.rawTransaction)
        
        print(f"Transaction sent: {tx_hash.hex()}")
        
        # Wait for confirmation
        receipt = self.w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Transaction confirmed in block: {receipt.blockNumber}")
        
        return receipt

    def get_my_chicken_ids(self, chicken_contract_address):
        """Get list of chicken NFT IDs owned by this address"""
        chicken_contract_address = self.w3.to_checksum_address(chicken_contract_address)
        
        # ERC721 ABI for basic NFT functions
        erc721_abi = [
            {
                "constant": True,
                "inputs": [{"name": "owner", "type": "address"}],
                "name": "balanceOf",
                "outputs": [{"name": "", "type": "uint256"}],
                "type": "function"
            },
            {
                "constant": True,
                "inputs": [{"name": "owner", "type": "address"}, {"name": "index", "type": "uint256"}],
                "name": "tokenOfOwnerByIndex",
                "outputs": [{"name": "", "type": "uint256"}],
                "type": "function"
            }
        ]
        
        try:
            chicken_contract = self.w3.eth.contract(
                address=chicken_contract_address,
                abi=erc721_abi
            )
            
            # Get number of NFTs owned
            balance = chicken_contract.functions.balanceOf(self.address).call()
            print(f"You own {balance} chicken NFTs")
            
            # Get all token IDs
            token_ids = []
            for i in range(balance):
                try:
                    token_id = chicken_contract.functions.tokenOfOwnerByIndex(self.address, i).call()
                    token_ids.append(token_id)
                    print(f"Found chicken NFT ID: {token_id}")
                except Exception as e:
                    print(f"Error getting token {i}: {e}")
                    # Add small delay to avoid rate limiting
                    import time
                    time.sleep(0.1)
            
            return token_ids
            
        except Exception as e:
            print(f"Error getting chicken IDs: {e}")
            print("Contract might not support ERC721Enumerable or may have different function names")
    def find_pokpok_contracts_from_history(self, limit=50):
        """Analyze your recent transactions to find PokPok contract addresses"""
        print(f"Analyzing recent transactions for address: {self.address}")
        
        try:
            # Get recent transactions
            latest_block = self.w3.eth.block_number
            
            # Check transactions from recent blocks
            pokpok_contracts = set()
            transaction_count = 0
            
            for block_num in range(latest_block - 1000, latest_block):
                try:
                    block = self.w3.eth.get_block(block_num, full_transactions=True)
                    
                    for tx in block.transactions:
                        if transaction_count >= limit:
                            break
                            
                        # Check if transaction involves your address
                        if tx['from'] == self.address or tx['to'] == self.address:
                            transaction_count += 1
                            
                            # Get transaction receipt for more details
                            receipt = self.w3.eth.get_transaction_receipt(tx['hash'])
                            
                            print(f"Transaction: {tx['hash'].hex()}")
                            print(f"  To: {tx['to']}")
                            print(f"  Value: {self.w3.from_wei(tx['value'], 'ether')} ETH")
                            print(f"  Gas Used: {receipt.gasUsed}")
                            
                            if tx['to']:
                                pokpok_contracts.add(tx['to'])
                            
                            # Check logs for contract interactions
                            for log in receipt.logs:
                                pokpok_contracts.add(log.address)
                            
                            print("---")
                            
                    if transaction_count >= limit:
                        break
                        
                except Exception as e:
                    continue  # Skip blocks that can't be accessed
            
            print(f"Found potential PokPok contracts:")
            for contract in pokpok_contracts:
                print(f"  {contract}")
                
            return list(pokpok_contracts)
            
        except Exception as e:
            print(f"Error analyzing transaction history: {e}")
            print("Try checking BaseScan manually or use a different approach")
            return []

    def get_staked_chicken_ids(self, staking_contract_address):
        """Get list of chicken NFT IDs that are currently staked"""
        staking_contract_address = self.w3.to_checksum_address(staking_contract_address)
        
        # Common staking contract functions - you may need to adjust these
        staking_abi = [
            {
                "constant": True,
                "inputs": [{"name": "user", "type": "address"}],
                "name": "getStakedTokens", 
                "outputs": [{"name": "", "type": "uint256[]"}],
                "type": "function"
            },
            {
                "constant": True,
                "inputs": [{"name": "user", "type": "address"}],
                "name": "stakedTokensOfOwner",
                "outputs": [{"name": "", "type": "uint256[]"}],
                "type": "function"
            }
        ]
        
        try:
            staking_contract = self.w3.eth.contract(
                address=staking_contract_address,
                abi=staking_abi
            )
            
            # Try different common function names
            try:
                staked_ids = staking_contract.functions.getStakedTokens(self.address).call()
                print(f"Found {len(staked_ids)} staked chickens via getStakedTokens")
                return staked_ids
            except:
                pass
                
            try:
                staked_ids = staking_contract.functions.stakedTokensOfOwner(self.address).call()
                print(f"Found {len(staked_ids)} staked chickens via stakedTokensOfOwner")
                return staked_ids
            except:
                pass
                
            print("Could not find staked tokens with common function names")
            return []
            
        except Exception as e:
            print(f"Error getting staked chicken IDs: {e}")
            return []

    def check_events_for_staked_chickens(self, staking_contract_address, from_block='latest'):
        """Look at blockchain events to find staked chickens"""
        staking_contract_address = self.w3.to_checksum_address(staking_contract_address)
        
        # Common staking event signatures
        stake_event_abi = [
            {
                "anonymous": False,
                "inputs": [
                    {"indexed": True, "name": "user", "type": "address"},
                    {"indexed": True, "name": "tokenId", "type": "uint256"}
                ],
                "name": "Staked",
                "type": "event"
            },
            {
                "anonymous": False,
                "inputs": [
                    {"indexed": True, "name": "user", "type": "address"},
                    {"indexed": True, "name": "tokenId", "type": "uint256"}
                ],
                "name": "Unstaked",
                "type": "event"
            }
        ]
        
        try:
            # Get events from the last 1000 blocks (adjust as needed)
            latest_block = self.w3.eth.block_number
            from_block_num = max(0, latest_block - 1000) if from_block == 'latest' else from_block
            
            # Create contract instance
            staking_contract = self.w3.eth.contract(
                address=staking_contract_address,
                abi=stake_event_abi
            )
            
            # Get staking events
            stake_filter = staking_contract.events.Staked.create_filter(
                fromBlock=from_block_num,
                argument_filters={'user': self.address}
            )
            
            unstake_filter = staking_contract.events.Unstaked.create_filter(
                fromBlock=from_block_num,
                argument_filters={'user': self.address}
            )
            
            staked_events = stake_filter.get_all_entries()
            unstaked_events = unstake_filter.get_all_entries()
            
            # Calculate currently staked tokens
            staked_tokens = set()
            for event in staked_events:
                staked_tokens.add(event.args.tokenId)
                
            for event in unstaked_events:
                staked_tokens.discard(event.args.tokenId)
            
            return list(staked_tokens)
            
        except Exception as e:
            print(f"Error checking events: {e}")
            return []

# Example usage
def main():
    # IMPORTANT: Never hardcode private keys in production!
    # Use environment variables or secure key management
    private_key = os.getenv('PRIVATE_KEY')  # Set this as environment variable
    
    if not private_key:
        print("Please set PRIVATE_KEY environment variable")
        return
    
    try:
        # Initialize interactor
        pokpok = PokPokInteractor(private_key)
        
        # Check PEGG balance
        pegg_balance = pokpok.get_pegg_balance()
        print(f"Current PEGG balance: {pegg_balance}")
        
        # Example: Find your staked chicken IDs
        # You'll need to replace these with actual contract addresses
        chicken_nft_contract = "0x13871995d62fEdfFAf2C5D26fca5739941e37572"
        # staking_contract = "0x..."      # Your staking/farm contract
        
        # Method 1: Get all chickens you own (if not staked)
        owned_chickens = pokpok.get_my_chicken_ids(chicken_nft_contract)
        # print(f"Chickens you own: {owned_chickens}")
        
        # Method 2: Get staked chickens from staking contract
        staked_chickens = pokpok.get_staked_chicken_ids(chicken_nft_contract)
        # print(f"Staked chickens: {staked_chickens}")
        
        # Method 3: Check events to find staked chickens
        # event_staked = pokpok.check_events_for_staked_chickens(staking_contract)
        # print(f"Staked chickens from events: {event_staked}")
        
        # Once you have the chicken IDs and contract addresses:
        # receipt = pokpok.claim_pegg_from_chickens(staking_contract, staked_chickens)
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

Connected to Base network
Account address: 0xa29E7239c202F8b39bFb322A15B0729FF3609514
Balance: 0.76109618750136735 ETH
PEGG Token Address: 0x82B0E1A2374EA0198f62A48B14FFaB53DB6C1E36
Current PEGG balance: 515.970587952544
You own 5 chicken NFTs
Found chicken NFT ID: 5615
Found chicken NFT ID: 5621
Found chicken NFT ID: 5622
Found chicken NFT ID: 6577
Found chicken NFT ID: 6601
Could not find staked tokens with common function names
